# Imports

In [1]:
import pandas as pd

# Load the data

In [2]:
data = pd.read_csv(
    "/shared_data/consensus_conference.csv", index_col="Unnamed: 0"
)
data

,name,job,company
1,Abi Subramanian,General Partner,Black Pine
2,Abie Cohen,Managing Partner,Centre Street Partners
3,Abigail Albright,Investorand Product,NaN
4,Abigail Castro,Developer Evangelist,Swirlds Labs
6,Abigail Johnson,COO,Sapphire Ventures
...,...,...,...
594,Zeno Saviour,Founder &amp; CEO,NaN
595,Zhuldyz Bakytzhan,Founder,Royal Crypto
596,Zia Matterblock,NaN,NaN
597,Zian Wang,Student,Uc Berkeley


# Tier 1 filter

## Either Partner or VC

In [3]:
# `Partner` in job.
mask1 = data["job"].str.contains("Partner", case=False, na=False)
print("Selecting partner=", mask1.sum())
# `Venture,Capital,VC` in company.
mask2 = data["company"].str.contains("venture|capital|VC", case=False, na=False)
print("Company count=", mask2.sum())
# Exclude LLP (lawyer firms).
mask3 = data["company"].str.contains("LLP", na=False)
print("Company count=", mask3.sum())

# Collect all Tier-1 contacts.
tier1_ease = data[(mask1 | mask2) & ~mask3]
tier1_ease = tier1_ease.drop_duplicates()
tier1_ease

Selecting partner= 560
Company count= 521
Company count= 88


,name,job,company
1,Abi Subramanian,General Partner,Black Pine
2,Abie Cohen,Managing Partner,Centre Street Partners
6,Abigail Johnson,COO,Sapphire Ventures
13,Abraham Choi,Partner / VP of Business Development,Energi
25,AdamBadrawi,"Partner, Comms",Framework Ventures
...,...,...,...
569,Wei Guo,Analyst,Whale Rock Capital Management
574,William Mccormick,Crypto Senior Partnerships,Robinhood
578,William Zapata,Investor,Thunder Bridge Capital
589,Yuki Yuminaga,Investment,Fenbushi Capital


## Partner and VC is obligatory

In [4]:
# `Partner` in job.
mask1 = data["job"].str.contains("Partner", case=False, na=False)
print("Selecting partner=", mask1.sum())
# `Venture,Capital,VC` in company.
mask2 = data["company"].str.contains("venture|capital|VC", case=False, na=False)
print("Company count=", mask2.sum())

# Collect all Tier-1 contacts.
tier1 = data[(mask1 & mask2)]
tier1 = tier1.drop_duplicates()
tier1

Selecting partner= 560
Company count= 521


,name,job,company
25,AdamBadrawi,"Partner, Comms",Framework Ventures
86,Adam William Davis,Managing Partner | CEO,C3BCNiFund Capital Management
193,Akash Singhania,Partner @Interblockcapital,Interblock Capital Partners
292,Alex Cheung,Founding Partner,Baboon VC
372,Alex Rogalski,Partner,LightSide Capital
...,...,...,...
14007,Zach Hawkins,Founding Partner,Lacuna Capital
14101,Zhiliang Li,Partner,Preon Capital
116,Cinta Putra,Venture Partner,VoicePunch.VC
361,Mario Merino,Managing Partner,Borderline Ventures


## Investment-related jobs

In [5]:
# `Investor` in job.
mask1 = data["job"].str.contains("investor", case=False, na=False)
print("Selecting Investor=", mask1.sum())
# Exclude IR positions.
mask2 = data["job"].str.contains("Relations", case=False, na=False)
# More investor-related jobs.
mask3 = data["job"].str.contains("investment", case=False, na=False)
print("Selecting Investment=", mask3.sum())

new_names = data[(mask1 | mask3) & ~mask2]
# Exclude previously found Tier-1 names.
new_names = new_names[~new_names["name"].isin(list(tier1["name"]))]
# Drop those with missing `company` field.
new_names = new_names[new_names["company"].notna()]
# Drop duplicates.
new_names = new_names.drop_duplicates(keep="last")
new_names

Selecting Investor= 163
Selecting Investment= 132


,name,job,company
149,Ahmed Alneyadi,Senior Investment Manager,ADIA
192,Akash Patel,Venture Capital Investments,Alphagrep
227,Alan Zeng,NFT Investor,AKINGS
537,Alina Chua,Investor,Mirana Ventures
546,Alison Hermann,Investment Manager,Trinity University
...,...,...,...
536,Thomas Mclaughlin,Chief Investment Officer,Coral DeFi
548,Trey Tull,Investor,Mask Mafia
558,Varun Ramakrishnan,Investment Team,General Atlantic
578,William Zapata,Investor,Thunder Bridge Capital


## Save results

In [6]:
new_names.to_csv("tier1.csv")

## Phantombuster format converter

In [7]:
def conver_to_phantombuster(df):
    ph_contacts = df["name"].map(str) + " " + df["company"].map(str)
    return ph_contacts

In [8]:
conver_to_phantombuster(new_names)

149                      Ahmed Alneyadi ADIA
192                    Akash Patel Alphagrep
227                         Alan Zeng AKINGS
537               Alina Chua Mirana Ventures
546        Alison Hermann Trinity University
                       ...                  
536             Thomas Mclaughlin Coral DeFi
548                     Trey Tull Mask Mafia
558      Varun Ramakrishnan General Atlantic
578    William Zapata Thunder Bridge Capital
589           Yuki Yuminaga Fenbushi Capital
Length: 257, dtype: object